# PUNN: Partition of Unity Neural Networks

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Akram-VU/PUNN/blob/main/PUNN_Demo.ipynb)

This notebook demonstrates the PUNN architecture and reproduces key experiments from the paper:

**"Partition of Unity Neural Networks for Interpretable Classification with Explicit Class Regions"**

---

## 1. Setup

Clone the repository and install dependencies.

In [ ]:
# Clone the PUNN repository
!git clone https://github.com/Akram-VU/PUNN.git
%cd PUNN

# Install dependencies
!pip install -q torch numpy matplotlib scikit-learn torchvision

In [ ]:
# Import PUNN models
import torch
import numpy as np
import matplotlib.pyplot as plt
from models import PUNN, MLP, PUNN_ShapeInformed

print("Setup complete!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## 2. Quick Demo: PUNN vs MLP

PUNN produces valid probabilities by construction (sum to 1), without softmax.

In [ ]:
# Create sample input
x = torch.randn(5, 10)  # 5 samples, 10 features

# PUNN model
punn = PUNN(input_dim=10, num_classes=3, gate_type='sigma', hidden_dim=32)
punn_probs = punn(x)

print("PUNN Output (probabilities by construction):")
print(punn_probs)
print(f"\nSum of probabilities: {punn_probs.sum(dim=1)}")
print(f"Parameters: {punn.count_parameters():,}")

## 3. Synthetic Experiments (Table 1)

Compare PUNN variants on Moons, Circles, XOR, and Helix datasets.

In [ ]:
# Run synthetic experiments (takes ~2 minutes)
!python experiments/synthetic_experiments.py

## 4. UCI Benchmark Experiments (Table 2)

Evaluate on standard UCI datasets: Iris, Wine, Breast Cancer, Digits, etc.

In [ ]:
# Run UCI experiments (takes ~5 minutes)
!python experiments/uci_experiments.py

## 5. MNIST Experiments (Table 3)

Large-scale evaluation on MNIST digit classification.

In [ ]:
# Run MNIST experiments (takes ~10 minutes with GPU)
!python experiments/mnist_experiments.py

## 6. Shape-Informed Gates (Table 4)

Demonstrate dramatic parameter reduction with geometric priors.

In [ ]:
# Run shape-informed experiments (takes ~3 minutes)
!python experiments/shape_informed_experiments.py

## 7. Custom Usage Example

Train PUNN on your own data.

In [ ]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.optim as optim

# Generate data
X, y = make_moons(n_samples=1000, noise=0.1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to tensors
X_train_t = torch.FloatTensor(X_train)
y_train_t = torch.LongTensor(y_train)
X_test_t = torch.FloatTensor(X_test)
y_test_t = torch.LongTensor(y_test)

# Create and train PUNN
model = PUNN(input_dim=2, num_classes=2, gate_type='sigma', hidden_dim=32)
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
for epoch in range(200):
    model.train()
    optimizer.zero_grad()
    probs = model(X_train_t)
    loss = -torch.log(probs[range(len(y_train_t)), y_train_t] + 1e-10).mean()
    loss.backward()
    optimizer.step()

# Evaluate
model.eval()
with torch.no_grad():
    test_probs = model(X_test_t)
    preds = test_probs.argmax(dim=1)
    accuracy = (preds == y_test_t).float().mean().item() * 100

print(f"Test Accuracy: {accuracy:.1f}%")
print(f"Parameters: {model.count_parameters():,}")

## 8. Visualize Decision Boundaries

In [ ]:
# Plot decision boundary
x_min, x_max = X_train[:, 0].min() - 0.5, X_train[:, 0].max() + 0.5
y_min, y_max = X_train[:, 1].min() - 0.5, X_train[:, 1].max() + 0.5
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),
                     np.linspace(y_min, y_max, 200))

grid = torch.FloatTensor(np.c_[xx.ravel(), yy.ravel()])
with torch.no_grad():
    Z = model(grid)[:, 1].numpy().reshape(xx.shape)

plt.figure(figsize=(8, 6))
plt.contourf(xx, yy, Z, levels=50, cmap='RdBu', alpha=0.8)
plt.contour(xx, yy, Z, levels=[0.5], colors='black', linewidths=2)
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap='RdBu',
            edgecolors='white', s=30)
plt.title('PUNN Decision Boundary (Moons Dataset)')
plt.colorbar(label='P(class=1)')
plt.show()

---

## Citation

```bibtex
@article{aldroubi2025punn,
  title={Partition of Unity Neural Networks for Interpretable Classification with Explicit Class Regions},
  author={Aldroubi, Akram},
  journal={},
  year={2025}
}
```